In [1]:
using Iterators
using Pipe
using Compat

macro printval(ee)
    ee_expr = @sprintf "%s" string(ee)
    esc(:(println($ee_expr," = ", $ee)))
end

macro pz(ee)
    ee_expr = @sprintf "%s" string(ee)
    esc(:(println($ee_expr,"\t\t",typeof($ee), "\t", size($ee))))
end

push!(LOAD_PATH, ".")
push!(LOAD_PATH, "../util/")

4-element Array{ByteString,1}:
 "/home/ubuntu/build/julia-master/usr/local/share/julia/site/v0.5"
 "/home/ubuntu/build/julia-master/usr/share/julia/site/v0.5"      
 "."                                                              
 "../util/"                                                       

In [ ]:
addprocs(1)

In [57]:
@everywhere using Distances

In [58]:
const START_MARKER = "**START**"
const END_MARKER = "**END**"

atis_data = open("atis_data.jsz","r") do fh
    deserialize(fh)    
end

@assert START_MARKER in atis_data["indexed_words"]
@assert START_MARKER in keys(atis_data["word_indexes"])
@assert END_MARKER in atis_data["indexed_words"]
@assert END_MARKER in keys(atis_data["word_indexes"])

In [59]:
@everywhere @inline function select_node{V<:AbstractVector}(edge_probs::V)
    ys, _, probs = findnz(edge_probs'')
    select_node(zip(ys,probs))
end

@everywhere function select_node(edge_probs)
    cutoff = rand()
    total = 0.0
    @assert(length(edge_probs)>0, "No options from point $x")
    for (y,prob) in edge_probs
        total+=prob
        if total>=cutoff
            return y
        end
    end
    
    @assert(total>=cutoff, "x=$x, total= $total, cutoff=$cutoff") #Should Never Reach this point
end


In [60]:
@everywhere function evaporate!{M<:AbstractArray}(τ::M, ρ)
    τ.*=(1.0-ρ) #Evaporation
end

    

In [61]:
# τ = Pheromone
# η = Prior
# α = pheremone weight
# β = prior weight
# ρ = pheremone evaporation

@everywhere function deposit!{M<:AbstractVector}(τ::M, path, fit)
    #Are You In or Out?  Double Crosser or Devout
    #Order actually means nothing in this problem
    #Cooccurance doesn't really many anything either
    #Cooccurance is a problem for the prior that need 
    #to be conerned with all answers not just this one
    for node in path
        @inbounds τ[node] += fit
    end
    τ
end

@everywhere function get_edge_probs(τ,η::Function, α, β)
    τ_scaled = τ.^α
    function get_prob(state)
        tot = τ_scaled .* η(state).^β
        tot./sum(tot)
    end
end

@everywhere @inline function select_node(edge_probs::Function, path)
    select_node(edge_probs(path))
end


In [181]:
#If you pass in a start index of greater than zero, then atre using a starting index, otherswise not
@everywhere function run_ant(edge_probs, end_index, start_index=0)
    path::Vector{Int} = start_index > 0 ? [start_index]  : [select_node(edge_probs, Int[])]
    while (path[end]!=end_index)
        push!(path, select_node(edge_probs, path))
    end
    path    
end


function optimise(fit_fun, τ, η, α, β,ρ, n_ants, n_gens, start_index, end_index, callback=Union{})
    assert(α>=1.0)
    assert(β>=1.0)
    assert(0.0<=ρ<1.0)
    
    max_fit::Float64 = -Inf;
    fittest_path::Vector{Int} = []
    for gen_ii in 1:n_gens
        if callback |> typeof <:Function
            callback(gen_ii, τ,fittest_path,max_fit)
        end
        print("*")
        edge_probs = get_edge_probs(τ,η, α, β)

        path_fits::Vector{Tuple{Vector{Int},Float64}} = map(1:n_ants) do ant_id
            path::Vector{Int} = run_ant(edge_probs, end_index,start_index)
            fit::Float64 = fit_fun(path)
            (path, fit)
        end
        
        evaporate!(τ,ρ)
        for (path, fit) in path_fits
            if fit>max_fit
                max_fit = fit
                fittest_path = path
                
                if fit==Inf #Perfect Fit
                    return fittest_path, max_fit
                end
            end
            deposit!(τ,path,fit)
        end

    end #Next Generation
    
    fittest_path, max_fit, τ
end


function optimise(fit_fun, n_dims::Int64, η::Function, α, β, ρ, n_ants, n_gens, start_index, end_index,callback=Union{})
    τ = [1.0 for nn in 1:n_dims]
    optimise(fit_fun, τ, η, α, β,ρ, n_ants, n_gens, start_index, end_index, callback)
end


function optimise{M<:AbstractMatrix}(fit_fun, η::M, α, β,ρ, n_ants, n_gens, start_index, end_index, callback=Union{})
    τ = ceil(η) #Since we take the element wise product with η, there is no point setting any elements that are zero in η 
    optimise(fit_fun, τ, η, α, β,ρ, n_ants, n_gens, start_index, end_index, callback)
end


optimise (generic function with 6 methods)

In [182]:
@fastmath function get_prior_fun(uni_occur,bi_occur,  prob_length, end_index)
    n_edges=length(uni_occur)
    function cooccur_prior(ws)
        if length(ws)==0
            uni_occur
        elseif length(ws)==1
            bi_occur[:,ws[1]]
        else
            (uni_occur + sum([bi_occur[:,w] for w in ws]))/(length(ws)+1) #Just average them cos its cheap
        end
    end
    
    function prior(ws)
        if  length(ws) > length(prob_length) #If it is longer than the tail of length distro, just force temrination
            force_end_pmf = zeros(n_edges) #This PMF forces the sentence to end, if it is returned
            @inbounds force_end_pmf[end_index]=1.0
            force_end_pmf
        else 
            #Either do not end (P=1-P_end) or end (P=P_end)
            prob_end = length(ws) == 0 ? 0.0 : prob_length[length(ws)]   #Chance to end at this length
            prob_occurance::Vector{Float64} = cooccur_prior(ws)
            
            choices_prob = (1.0-prob_end).*prob_occurance
            @inbounds choices_prob[end_index]=prob_end
            choices_prob
        end 
    end
    prior
end

function get_prior_fun(data::Dict)
    get_prior_fun(data["unioccur"], data["bioccur"], data["length_prob"], data["word_indexes"][END_MARKER])
end

get_prior_fun (generic function with 2 methods)

In [183]:
function lookup_sowe(data, sent)
    sum([data["LL"][:,data["word_indexes"][word]] for word in split(sent)]) 
end
function lookup_words(data, path)
    [data["indexed_words"][ii] for ii in path]
end


lookup_words (generic function with 1 method)

In [184]:
function built_optimise(data::Dict, target_sent::AbstractString, α, β, ρ, n_ants, n_gens, log=False)
    target::Vector{Float64} = lookup_sowe(data,target_sent)
    built_optimise(data, target, α, β,ρ, n_ants, n_gens, log)
end

function built_optimise(data::Dict, target::AbstractVector, α, β,ρ, n_ants, n_gens, log=False)
    LL =data["LL"]
    indexed_words=data["indexed_words"]
    @inline function fitness(path)
        #Fitter is larger
        @inbounds path_end = sum([LL[:,ii] for ii in path]) 
        (1.0/euclidean(path_end, target))/length(path)
    end
    
    τlog=[]
    if log
        log = (gen_ii, τ,fittest_path,max_fit) -> begin
            print("oGen: $gen_ii - fit: $max_fit ")
            println(lookup_words(data, fittest_path))
            push!(τlog,τ)
        end
    end
    
    
    start_index = 0 #not in use #data["word_indexes"][START_MARKER]
    end_index = data["word_indexes"][END_MARKER]
    
    prior = get_prior_fun(data)
    n_dims = size(LL,2)
    path, score, τ = optimise(fitness,n_dims, prior, α, β,ρ, n_ants, n_gens, start_index, end_index,log)
    push!(τlog,τ)
    score,lookup_words(data, path), τlog
end

built_optimise (generic function with 4 methods)

In [188]:
score, words, τlog = built_optimise(atis_data,"show prices", 2, 1, 0.3, 10000,20, true);
words

oGen: 1 - fit: -Inf Any[]
*oGen: 2 - fit: 0.500000015357572 Any["prices","**END**"]
*oGen: 3 - fit: 0.500000015357572 Any["prices","**END**"]
*oGen: 4 - fit: 0.500000015357572 Any["prices","**END**"]
*oGen: 5 - fit: 0.500000015357572 Any["prices","**END**"]
*oGen: 6 - fit: 0.500000015357572 Any["prices","**END**"]
*oGen: 7 - fit: 0.500000015357572 Any["prices","**END**"]
*oGen: 8 - fit: 0.500000015357572 Any["prices","**END**"]
*oGen: 9 - fit: 0.500000015357572 Any["prices","**END**"]
*oGen: 10 - fit: 0.500000015357572 Any["prices","**END**"]
*oGen: 11 - fit: 0.500000015357572 Any["prices","**END**"]
*oGen: 12 - fit: 0.500000015357572 Any["prices","**END**"]
*oGen: 13 - fit: 0.500000015357572 Any["prices","**END**"]
*oGen: 14 - fit: 0.500000015357572 Any["prices","**END**"]
*oGen: 15 - fit: 0.500000015357572 Any["prices","**END**"]
*oGen: 16 - fit: 0.500000015357572 Any["prices","**END**"]
*oGen: 17 - fit: 0.500000015357572 Any["prices","**END**"]
*oGen: 18 - fit: 0.500000015357572 Any

2-element Array{Any,1}:
 "prices" 
 "**END**"

["prices","**END**"]
*

In [173]:
?read

search: 

```
read(stream, type, dims)
```

Read a series of values of the given type from a stream, in canonical binary representation. `dims` is either a tuple or a series of integer arguments specifying the size of `Array` to return.

```
read(stream, type)
```

Read a value of the given type from a stream, in canonical binary representation.


read read! readdlm readdir readcsv readall readlink readline readuntil



In [187]:
τlog

21-element Array{Any,1}:
 [46.7663,26.5188,27.4646,78.9656,194.996,9.37693,1.0,1.49467,697.28,10.1376,1.96954,2.97695,4.64213,3.90968,910.828,3.27748,31.7549,12.5522,4.9833,5.05401,4.14627,7.25856,3.53182,1.49138,1.1662,1.82364,1.22946,1.0,1.00115,1.23653,1.55497,1.99254,1.28194,1.0,15.4282,1.52031,39.7313,5.23856,21.321,15.0261,2.08304,11.7102,18.4472,1.41604,21.6727,10.2798,16.7788,8.94804,5.95069,3.71325,3.21766,2.16856,3.23324,2.50818,2.29415,1.35198,2.79054,18.7805,4.33497,1.96889,1.35831,18.4587,1.34359,172.491,1.83588,1.81942,20.3648,3.09795,3.7808,4.52809,1.0,2.63629,1.96961,7.5595,1.0,11.162,1.2205,4.4343,1.0,1.65523,3.07528,2.83308,1.06844,4.40998,2.27677,3.42165,1.22945,1.45624,1.85856,4.4191,1.51979,1.29424,1.71397,5.85306,10.594,3.83275,12.4167,1.52779,1.3727,4.3821,1.31063,13.0078,1.04055,2.69858,1.09705,2.43835,4.60724,6.4625,1.0,6.62157,1.0,1.0,2.88373,1.07907,2.56228,16.9582,1.0,1.28089,1.13863,9.14995,1.05752,84.9876,14.5854,1.08902,3.10079,2.02291,2.21525,7.1037,1.04

In [189]:
using UnicodePlots
using IJulia

for τ in τlog
    nw = zip(τ,atis_data["indexed_words"]) |> collect
    
    freq,name = zip(select!(nw, 1:min(20,length(nw)), by=fw->-fw[1])...) |> collect
    IJulia.display(barplot(name|> collect,freq|> collect))
end


           ┌────────────────────────────────────────┐ 
   **END** │▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪ 63754.4055571627 │ 
        to │▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪ 63725.09929838031│ 
       the │ 1244.448035546863                      │ 
      from │ 1073.452828172626                      │ 
   flights │ 449.50587729422693                     │ 
        on │ 257.847346052582                       │ 
    flight │ 248.33740471709615                     │ 
      what │ 243.934931375034                       │ 
      show │ 153.20548697927248                     │ 
        is │ 121.79656630831458                     │ 
         a │ 119.19192339861023                     │ 
        me │ 108.0074846246886                      │ 
    boston │ 87.21517725939005                      │ 
         i │ 82.26161761718998                      │ 
    denver │ 67.5077905498013                       │ 
        of │ 67.21044305076161                      │ 
        in │ 57.69959714798049                      │ 
       san

           ┌────────────────────────────────────────┐ 
   **END** │▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪ 63754.4055571627 │ 
        to │▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪ 63725.09929838031│ 
       the │ 1244.448035546863                      │ 
      from │ 1073.452828172626                      │ 
   flights │ 449.50587729422693                     │ 
        on │ 257.847346052582                       │ 
    flight │ 248.33740471709615                     │ 
      what │ 243.934931375034                       │ 
      show │ 153.20548697927248                     │ 
        is │ 121.79656630831458                     │ 
         a │ 119.19192339861023                     │ 
        me │ 108.0074846246886                      │ 
    boston │ 87.21517725939005                      │ 
         i │ 82.26161761718998                      │ 
    denver │ 67.5077905498013                       │ 
        of │ 67.21044305076161                      │ 
        in │ 57.69959714798049                      │ 
       san

           ┌────────────────────────────────────────┐ 
   **END** │▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪ 63754.4055571627 │ 
        to │▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪ 63725.09929838031│ 
       the │ 1244.448035546863                      │ 
      from │ 1073.452828172626                      │ 
   flights │ 449.50587729422693                     │ 
        on │ 257.847346052582                       │ 
    flight │ 248.33740471709615                     │ 
      what │ 243.934931375034                       │ 
      show │ 153.20548697927248                     │ 
        is │ 121.79656630831458                     │ 
         a │ 119.19192339861023                     │ 
        me │ 108.0074846246886                      │ 
    boston │ 87.21517725939005                      │ 
         i │ 82.26161761718998                      │ 
    denver │ 67.5077905498013                       │ 
        of │ 67.21044305076161                      │ 
        in │ 57.69959714798049                      │ 
       san

           ┌────────────────────────────────────────┐ 
   **END** │▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪ 63754.4055571627 │ 
        to │▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪ 63725.09929838031│ 
       the │ 1244.448035546863                      │ 
      from │ 1073.452828172626                      │ 
   flights │ 449.50587729422693                     │ 
        on │ 257.847346052582                       │ 
    flight │ 248.33740471709615                     │ 
      what │ 243.934931375034                       │ 
      show │ 153.20548697927248                     │ 
        is │ 121.79656630831458                     │ 
         a │ 119.19192339861023                     │ 
        me │ 108.0074846246886                      │ 
    boston │ 87.21517725939005                      │ 
         i │ 82.26161761718998                      │ 
    denver │ 67.5077905498013                       │ 
        of │ 67.21044305076161                      │ 
        in │ 57.69959714798049                      │ 
       san

           ┌────────────────────────────────────────┐ 
   **END** │▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪ 63754.4055571627 │ 
        to │▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪ 63725.09929838031│ 
       the │ 1244.448035546863                      │ 
      from │ 1073.452828172626                      │ 
   flights │ 449.50587729422693                     │ 
        on │ 257.847346052582                       │ 
    flight │ 248.33740471709615                     │ 
      what │ 243.934931375034                       │ 
      show │ 153.20548697927248                     │ 
        is │ 121.79656630831458                     │ 
         a │ 119.19192339861023                     │ 
        me │ 108.0074846246886                      │ 
    boston │ 87.21517725939005                      │ 
         i │ 82.26161761718998                      │ 
    denver │ 67.5077905498013                       │ 
        of │ 67.21044305076161                      │ 
        in │ 57.69959714798049                      │ 
       san

...

           ┌────────────────────────────────────────┐ 
   **END** │▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪ 63754.4055571627 │ 
        to │▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪ 63725.09929838031│ 
       the │ 1244.448035546863                      │ 
      from │ 1073.452828172626                      │ 
   flights │ 449.50587729422693                     │ 
        on │ 257.847346052582                       │ 
    flight │ 248.33740471709615                     │ 
      what │ 243.934931375034                       │ 
      show │ 153.20548697927248                     │ 
        is │ 121.79656630831458                     │ 
         a │ 119.19192339861023                     │ 
        me │ 108.0074846246886                      │ 
    boston │ 87.21517725939005                      │ 
         i │ 82.26161761718998                      │ 
    denver │ 67.5077905498013                       │ 
        of │ 67.21044305076161                      │ 
        in │ 57.69959714798049                      │ 
       san

           ┌────────────────────────────────────────┐ 
   **END** │▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪ 63754.4055571627 │ 
        to │▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪ 63725.09929838031│ 
       the │ 1244.448035546863                      │ 
      from │ 1073.452828172626                      │ 
   flights │ 449.50587729422693                     │ 
        on │ 257.847346052582                       │ 
    flight │ 248.33740471709615                     │ 
      what │ 243.934931375034                       │ 
      show │ 153.20548697927248                     │ 
        is │ 121.79656630831458                     │ 
         a │ 119.19192339861023                     │ 
        me │ 108.0074846246886                      │ 
    boston │ 87.21517725939005                      │ 
         i │ 82.26161761718998                      │ 
    denver │ 67.5077905498013                       │ 
        of │ 67.21044305076161                      │ 
        in │ 57.69959714798049                      │ 
       san

           ┌────────────────────────────────────────┐ 
   **END** │▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪ 63754.4055571627 │ 
        to │▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪ 63725.09929838031│ 
       the │ 1244.448035546863                      │ 
      from │ 1073.452828172626                      │ 
   flights │ 449.50587729422693                     │ 
        on │ 257.847346052582                       │ 
    flight │ 248.33740471709615                     │ 
      what │ 243.934931375034                       │ 
      show │ 153.20548697927248                     │ 
        is │ 121.79656630831458                     │ 
         a │ 119.19192339861023                     │ 
        me │ 108.0074846246886                      │ 
    boston │ 87.21517725939005                      │ 
         i │ 82.26161761718998                      │ 
    denver │ 67.5077905498013                       │ 
        of │ 67.21044305076161                      │ 
        in │ 57.69959714798049                      │ 
       san

           ┌────────────────────────────────────────┐ 
   **END** │▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪ 63754.4055571627 │ 
        to │▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪ 63725.09929838031│ 
       the │ 1244.448035546863                      │ 
      from │ 1073.452828172626                      │ 
   flights │ 449.50587729422693                     │ 
        on │ 257.847346052582                       │ 
    flight │ 248.33740471709615                     │ 
      what │ 243.934931375034                       │ 
      show │ 153.20548697927248                     │ 
        is │ 121.79656630831458                     │ 
         a │ 119.19192339861023                     │ 
        me │ 108.0074846246886                      │ 
    boston │ 87.21517725939005                      │ 
         i │ 82.26161761718998                      │ 
    denver │ 67.5077905498013                       │ 
        of │ 67.21044305076161                      │ 
        in │ 57.69959714798049                      │ 
       san

           ┌────────────────────────────────────────┐ 
   **END** │▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪ 63754.4055571627 │ 
        to │▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪▪ 63725.09929838031│ 
       the │ 1244.448035546863                      │ 
      from │ 1073.452828172626                      │ 
   flights │ 449.50587729422693                     │ 
        on │ 257.847346052582                       │ 
    flight │ 248.33740471709615                     │ 
      what │ 243.934931375034                       │ 
      show │ 153.20548697927248                     │ 
        is │ 121.79656630831458                     │ 
         a │ 119.19192339861023                     │ 
        me │ 108.0074846246886                      │ 
    boston │ 87.21517725939005                      │ 
         i │ 82.26161761718998                      │ 
    denver │ 67.5077905498013                       │ 
        of │ 67.21044305076161                      │ 
        in │ 57.69959714798049                      │ 
       san

In [179]:
τlog

1-element Array{Any,1}:
 [1,2]

In [75]:
[lookup_sowe(atis_data, "washington washington") lookup_sowe(atis_data, "washington")]


300x2 Array{Float64,2}:
 -0.129684    -0.0648421 
  0.0208328    0.0104164 
 -0.036197    -0.0180985 
  0.253119     0.126559  
 -0.0312492   -0.0156246 
 -0.121872    -0.060936  
 -0.132288    -0.0661442 
  0.0356762    0.0178381 
 -0.129684    -0.0648421 
 -0.00445952  -0.00222976
 -0.0653629   -0.0326815 
 -0.235411    -0.117705  
 -0.0864561   -0.0432281 
  ⋮                      
  0.0651025    0.0325513 
 -0.0150387   -0.00751934
 -0.0388011   -0.0194006 
  0.0630192    0.0315096 
  0.193745     0.0968726 
  0.0460926    0.0230463 
  0.072394     0.036197  
 -0.11406     -0.0570298 
 -0.148955    -0.0744773 
 -0.127601    -0.0638005 
  0.062238     0.031119  
 -0.0403636   -0.0201818 

In [30]:
atis_data["unioccur"][atis_data["word_indexes"]["show"]]

0.014320531528454779

In [ ]:
atis_data["unioccur"][atis_data["word_indexes"]["showed"]]

In [ ]:
#Fast Splitting
subset_indexes_for_len = Dict{Int,Vector{Vector{Bool}}}()
function nonempty_subset_splits{T}(xs::Vector{T})
    K = length(xs)
    ss ::Vector{Vector{Bool}} = if haskey(subset_indexes_for_len,K)
            subset_indexes_for_len[K]
         else 
            subset_indexes_for_len[K] = Vector{Bool}[collect(pr) for (_,pr) in  zip(1:(K^2)/2, product([(true,false) for k in 1:K]...))][2:end]
            subset_indexes_for_len[K]
         end

    Tuple{Vector{T},Vector{T}}[(xs[bb],xs[~bb]) for bb in ss]
end

function nonempty_subset_splits(xs)
    nonempty_subset_splits(collect(xs))
end



function pairwise(pp)
    Task() do 
        for ii in 2:2:length(pp)
            produce((pp[ii-1],pp[ii]))
        end
        if isodd(length(pp))
            produce(pp[end]) 
        end
    end
end

_partions_of_pairs_indexes=Dict{Vector{Int64},Vector{Vector{Int64}}}()
function partions_of_pairs_indexes{Int64}(xs::Vector{Int64})
    if !haskey(_partions_of_pairs_indexes, xs)
        _partions_of_pairs_indexes[xs] = 
            if length(xs)<=2
                Vector{Int64}[xs]
            else
                rets = Vector{Vector{Int64}}()
                sizehint!(rets, length(xs)^div(length(xs),2))
                for i1 in 1:length(xs-1)
                    for i2 in i1+1:length(xs)
                        head = (xs[[i1,i2]])
                        tails = partions_of_pairs_indexes(xs[[1:i1-1;i1+1:i2-1;i2+1:end]])

                        for tail in tails
                                push!(rets, [head; tail])
                            end
                        end
                end
                rets
            end
    end
    _partions_of_pairs_indexes[xs] 
end

function partions_of_pairs(xs::Vector{Int64})
    partions_of_pairs_indexes(xs)
end
#Fast Splitting
subset_indexes_for_len = Dict{Int,Vector{Vector{Bool}}}()
function nonempty_subset_splits{T}(xs::Vector{T})
    K = length(xs)
    ss ::Vector{Vector{Bool}} = if haskey(subset_indexes_for_len,K)
            subset_indexes_for_len[K]
         else 
            subset_indexes_for_len[K] = Vector{Bool}[collect(pr) for (_,pr) in  zip(1:(K^2)/2, product([(true,false) for k in 1:K]...))][2:end]
            subset_indexes_for_len[K]
         end

    Tuple{Vector{T},Vector{T}}[(xs[bb],xs[~bb]) for bb in ss]
end

function nonempty_subset_splits(xs)
    nonempty_subset_splits(collect(xs))
end



function pairwise(pp)
    Task() do 
        for ii in 2:2:length(pp)
            produce((pp[ii-1],pp[ii]))
        end
        if isodd(length(pp))
            produce(pp[end]) 
        end
    end
end

_partions_of_pairs_indexes=Dict{Vector{Int64},Vector{Vector{Int64}}}()
function partions_of_pairs_indexes{Int64}(xs::Vector{Int64})
    if !haskey(_partions_of_pairs_indexes, xs)
        _partions_of_pairs_indexes[xs] = 
            if length(xs)<=2
                Vector{Int64}[xs]
            else
                rets = Vector{Vector{Int64}}()
                sizehint!(rets, length(xs)^div(length(xs),2))
                for i1 in 1:length(xs-1)
                    for i2 in i1+1:length(xs)
                        head = (xs[[i1,i2]])
                        tails = partions_of_pairs_indexes(xs[[1:i1-1;i1+1:i2-1;i2+1:end]])

                        for tail in tails
                                push!(rets, [head; tail])
                            end
                        end
                end
                rets
            end
    end
    _partions_of_pairs_indexes[xs] 
end

function partions_of_pairs(xs::Vector{Int64})
    partions_of_pairs_indexes(xs)
end


In [ ]:
using DataStructures
using DataStructuresExtended


@fastmath function get_prior_fun(uni_occur, bi_joint, prob_length, end_index)
    n_edges=length(uni_occur)
    bi_marginal = sum(bi_joint,1)
    bi_conditional = bi_joint./bi_marginal
  
    
    @inline function joint(aa::Int64)
        @inbounds ret = bi_marginal[aa]
        ret
    end
    
    @inline function joint(aa::Int64,bb::Int64)
        @inbounds ret = bi_joint[aa, bb]
        ret
    end
    
    joint_cache_stats = Dict([("hits",0), ("misses",0)])
    joint_cache = Dict{Accumulator{Int64,Int64}, Float64}() #TODO Should Use a multiset instead to index
    @inline function joint(xxs...) 
        @assert(length(xxs)>=3)
        xxs_bag = counter(Int64, xxs) 
        if !(haskey(joint_cache,xxs_bag))
            joint_cache_stats["misses"]+=1
            #Full calc
            joint_cache[xxs_bag] = mean(Float64[(joint(aas...)*joint(bbs...)) for (aas,bbs) in  nonempty_subset_splits(xxs)])
            
            
        else
            joint_cache_stats["hits"]+=1
        end
        joint_cache[xxs_bag]
    end
    
    conditional_occurance::Function
    @inbounds function conditional_occurance(ws::Vector{Int64})
        if length(ws) == 0
            uni_occur
        elseif length(ws)==1
            bi_conditional[:,ws[1]]
        else
            Float64[joint(aa,ws...) for aa in 1:n_edges]./joint(ws...)
        end
    end
    
    function prior(ws)
        if  length(ws) > length(prob_length) #If it is longer than the tail of length distro, just force temrination
            force_end_pmf = zeros(n_edges) #This PMF forces the sentence to end, if it is returned
            @inbounds force_end_pmf[end_index]=1.0
            force_end_pmf
        else 
            #Either do not end (P=1-P_end) or end (P=P_end)
            #prob_end = prob_length[length(ws)]   #Chance to end at this length
            prob_end=0.0
            prob_occurance::Vector{Float64} = conditional_occurance(ws)
            
            choices_prob = (1.0-prob_end).*prob_occurance
            #@inbounds choices_prob[end_index]=prob_end
            choices_prob
        end 
    end
    prior,joint_cache_stats
end

function get_prior_fun(data::Dict)
    get_prior_fun(data["unioccur"],data["bioccur"], data["length_prob"], data["word_indexes"][END_MARKER])
end

In [ ]:
atis_prior,atis_prior_cache_stats = get_prior_fun(atis_data)
r = @time atis_prior(Int64[1,2,5,4,5,6,7]) #9.335965 seconds
println(atis_prior_cache_stats)
r

In [ ]:
optimise(atis_data,"what is the shortest flight", 1, 1, 0.1, 10000,20)

In [ ]:
optimise(atis_data,"what is the first flight from washington to washington", 1, 1, 0.1, 10000,20)

In [ ]:
optimise(atis_data,"what are the ground transport options", 1, 1, 0.1, 1000,20)

In [ ]:
optimise(atis_data,"where can i go", 1, 1, 0.1, 1000,20)

In [ ]:
atis_data["word_indexes"][START_MARKER]
    

In [ ]:
`git push""` |> run

In [ ]:
`git commit -m="" -a` |> run